## 5-Minute Quick Demo of Diaspora Action Provider Features

In [1]:
"""Import necessary libraries for Demo."""
from __future__ import annotations

import base64
import json
import os
import random
import string
import time
import uuid

import globus_sdk
import globus_sdk.scopes
from diaspora_event_sdk import block_until_ready
from diaspora_event_sdk import Client
from diaspora_event_sdk import KafkaConsumer
from diaspora_event_sdk import KafkaProducer
from diaspora_event_sdk.version import __version__ as diaspora_sdk_version
from globus_sdk import __version__ as globus_sdk_version

In [36]:
# c = Client() # re-auth if there's something wrong with the access token
# c.logout()

In [3]:
c = Client()
print('Globus SDK version:', globus_sdk_version)
print('Diaspora Event SDK version:', diaspora_sdk_version)
print("User's OpenID:", c.subject_openid)

Please authenticate with Globus here:
------------------------------------
https://auth.globus.org/v2/oauth2/authorize?client_id=c5d4fab4-7f0d-422e-b0c8-5c74329b52fe&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2F2b9d2f5c-fa32-45b5-875b-b24cd343b917%2Faction_all+openid&state=_default&response_type=code&code_challenge=FQub5-j_GCfGlLIPU2wrRhazxpI_PaVIZPHXO_SCsqk&code_challenge_method=S256&access_type=offline&prefill_named_grant=HaochenPans-Laptop.local&prompt=login
------------------------------------

Globus SDK version: 3.41.0
Diaspora Event SDK version: 0.3.2
User's OpenID: e2a8169b-feef-4d56-8eba-ab12747bee03


In [30]:
# ID of this tutorial notebook as registered with Globus Auth
CLIENT_ID = 'f794186b-f330-4595-b6c6-9c9d3e903e47'

# Do a native app authentication flow to get tokens that allow us
# to interact with the Globus Flows service

scopes = [
    'openid',
    'profile',
    'email',
    globus_sdk.FlowsClient.scopes.manage_flows,
    globus_sdk.FlowsClient.scopes.run_manage,
]
native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
native_auth_client.oauth2_start_flow(requested_scopes=scopes)
print(f'Login Here:\n\n{native_auth_client.oauth2_get_authorize_url()}')

auth_code = input('Authorization Code: ')
response = native_auth_client.oauth2_exchange_code_for_tokens(auth_code)

tokens = response.by_resource_server
print(json.dumps(tokens, indent=2))

flows_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens['flows.globus.org']['access_token'],
)
flows_client = globus_sdk.FlowsClient(authorizer=flows_authorizer)

Login Here:

https://auth.globus.org/v2/oauth2/authorize?client_id=f794186b-f330-4595-b6c6-9c9d3e903e47&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+https%3A%2F%2Fauth.globus.org%2Fscopes%2Feec9b274-0c81-4334-bdc2-54e90e689b9a%2Fmanage_flows+https%3A%2F%2Fauth.globus.org%2Fscopes%2Feec9b274-0c81-4334-bdc2-54e90e689b9a%2Frun_manage&state=_default&response_type=code&code_challenge=pDwoUw7A7uuF1FAPW1O3WE6g_FtmpAHUrPHXFEFpfjY&code_challenge_method=S256&access_type=online
{
  "auth.globus.org": {
    "scope": "profile openid email",
    "access_token": "Ag4b946mNPd02v9W7zoD25x5E00D2QPeGDBJyxOBPGDvMy1NEWt7CDqvKWdwygjOq3Q13ElzB25wVKIvevbPpio76jWHrMe3wugnl0Y",
    "refresh_token": null,
    "token_type": "Bearer",
    "expires_at_seconds": 1724987940,
    "resource_server": "auth.globus.org"
  },
  "flows.globus.org": {
    "scope": "https://auth.globus.org/scopes/eec9b274-0c81-4334-bdc2-54e90e689b9a/run_manage https://auth.globus.org/scopes/eec9

In [33]:
topic1 = 'topic1-' + c.subject_openid[-12:]
print('Topic name:', topic1)
print(c.register_topic(topic1))

Topic name: topic1-ab12747bee03
{
  "status": "no-op",
  "message": "Principal e2a8169b-feef-4d56-8eba-ab12747bee03 already has access."
}


In [34]:
# Create an Auth client so we can look up identities
auth_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens['auth.globus.org']['access_token'],
)
ac = globus_sdk.AuthClient(authorizer=auth_authorizer)

# Get the user's primary identity
primary_identity = ac.oauth2_userinfo()
identity_id = primary_identity['sub']

print(f"Username: {primary_identity['preferred_username']}")
print(f'ID: {identity_id}')
print('Topic to produce/consume:', topic1)

Username: haochenpan@uchicago.edu
ID: e2a8169b-feef-4d56-8eba-ab12747bee03
Topic to produce/consume: topic1-ab12747bee03


### 1. Produce events to AP with event keys

In [35]:
action_url = 'https://diaspora-action-provider.ml22sevubfnks.us-east-1.cs.amazonlightsail.com/'

flow_definition = {
    'Comment': 'Publish messages to Diaspora Event Fabric',
    'StartAt': 'PublishMessages',
    'States': {
        'PublishMessages': {
            'Comment': 'Send messages to a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'msgs.$': '$.input.msgs',
                'keys.$': '$.input.keys',
            },
            'ResultPath': '$.PublishMessages',
            'End': True,
        },
    },
}

In [36]:
flow_title = f'Diapora-AP-Flow-{str(uuid.uuid4())[:4]}'
flow = flows_client.create_flow(
    title=flow_title,
    definition=flow_definition,
    input_schema={},
)
flow_id = flow['id']
flow_scope = globus_sdk.SpecificFlowClient(flow_id).scopes.make_mutable('user')
print(f"Successfully created flow: '{flow_title} (ID: {flow_id})")
print(f'View the flow in the Web App: https://app.globus.org/flows/{flow_id}')

if flow_id not in tokens:
    # Do a native app authentication flow and get tokens that
    # include the newly deployed flow scope
    native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
    native_auth_client.oauth2_start_flow(requested_scopes=flow_scope)
    print(f'Login Here:\n\n{native_auth_client.oauth2_get_authorize_url()}')

    # Authenticate and come back with your authorization code;
    # paste it into the prompt below.
    auth_code = input('Authorization Code: ')
    token_response = native_auth_client.oauth2_exchange_code_for_tokens(
        auth_code,
    )

    # Save the new token in a place where the flows client can retrieve it.
    tokens[flow_id] = token_response.by_resource_server[flow_id]

    # These are the saved scopes for the flow
    print(json.dumps(tokens, indent=2))

Successfully created flow: 'Diapora-AP-Flow-a6c9 (ID: 0b23af30-5d20-4187-858d-aa921135475f)
View the flow in the Web App: https://app.globus.org/flows/0b23af30-5d20-4187-858d-aa921135475f
Login Here:

https://auth.globus.org/v2/oauth2/authorize?client_id=f794186b-f330-4595-b6c6-9c9d3e903e47&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2F0b23af30-5d20-4187-858d-aa921135475f%2Fflow_0b23af30_5d20_4187_858d_aa921135475f_user&state=_default&response_type=code&code_challenge=xSZgMzmssD0tz837AsDTdiaGFujqmArooXFsyC8-g6w&code_challenge_method=S256&access_type=online
{
  "auth.globus.org": {
    "scope": "profile openid email",
    "access_token": "Ag4b946mNPd02v9W7zoD25x5E00D2QPeGDBJyxOBPGDvMy1NEWt7CDqvKWdwygjOq3Q13ElzB25wVKIvevbPpio76jWHrMe3wugnl0Y",
    "refresh_token": null,
    "token_type": "Bearer",
    "expires_at_seconds": 1724987940,
    "resource_server": "auth.globus.org"
  },
  "flows.globus.org": {
    "scope": "http

In [37]:
flow_input = {
    'input': {
        'action': 'produce',
        'topic': topic1,
        'msgs': [
            {'content1': 'argonne national laboratory'},
            {'content2': 'university of chicago'},
            {'content3': 'johns hopkins university'},
        ],
        'keys': [
            'my-key-123',
            'my-key-456',
            'my-key-789',
        ],
    },
}

In [38]:
# Get a client for the flow
specific_flow_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens[flow_id]['access_token'],
)
print(tokens[flow_id]['access_token'])
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    authorizer=specific_flow_authorizer,
)

# Run the flow
# Set a descriptive label for this flow run
run_label = f"Diaspora AP Flow by {primary_identity['preferred_username']}"
run = specific_flow_client.run_flow(
    body=flow_input,
    label=run_label,
    tags=['tutorial', 'diaspora'],
)

# Get run details
run_id = run['run_id']
run_status = run['status']
print('This flow can be monitored in the Web App:')
print(f'https://app.globus.org/runs/{run_id}')
print(f'Flow run started with ID: {run_id} - Status: {run_status}')

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')

# Run completed
print(json.dumps(run.data, indent=2))

AgW3XqdVa9kqX9JP7wQV7k6jV7XxrPnDBnwoPODG74Bx022z18igCko6j75VEOxx1xYy0nXpV3jV5xCl1lw4miBPMxO
This flow can be monitored in the Web App:
https://app.globus.org/runs/93c6c5ae-4d35-4783-8590-df12ae5a0d04
Flow run started with ID: 93c6c5ae-4d35-4783-8590-df12ae5a0d04 - Status: ACTIVE
Run status: ACTIVE
Run status: SUCCEEDED
{
  "run_id": "93c6c5ae-4d35-4783-8590-df12ae5a0d04",
  "flow_id": "0b23af30-5d20-4187-858d-aa921135475f",
  "flow_title": "Diapora-AP-Flow-a6c9",
  "flow_last_updated": "2024-08-28T03:20:12.196355+00:00",
  "start_time": "2024-08-28T03:20:35.877528+00:00",
  "completion_time": "2024-08-28T03:20:40.968000+00:00",
  "status": "SUCCEEDED",
  "display_status": "SUCCEEDED",
  "details": {
    "code": "FlowSucceeded",
    "output": {
      "input": {
        "keys": [
          "my-key-123",
          "my-key-456",
          "my-key-789"
        ],
        "msgs": [
          {
            "content1": "argonne national laboratory"
          },
          {
            "content

### 2. Blocking consume events from AP with specific prefix after a timestamp

In [39]:
flow_definition_b = {
    'Comment': 'Consume messages to Diaspora Event Fabric',
    'StartAt': 'ConsumeMessages',
    'States': {
        'ConsumeMessages': {
            'Comment': 'Receive messages from a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'ts.$': '$.input.ts',
                # 'group_id.$': '$.input.group_id',
                'filters.$': '$.input.filters',
            },
            'ResultPath': '$.ConsumeMessages',
            'End': True,
        },
    },
}

flows_client.update_flow(
    flow_id,
    definition=flow_definition_b,
    input_schema={},
)

GlobusHTTPResponse({"id": "0b23af30-5d20-4187-858d-aa921135475f", "definition": {"Comment": "Consume messages to Diaspora Event Fabric", "StartAt": "ConsumeMessages", "States": {"ConsumeMessages": {"Comment": "Receive messages from a specified topic in Diaspora", "Type": "Action", "ActionUrl": "https://diaspora-action-provider.ml22sevubfnks.us-east-1.cs.amazonlightsail.com/", "Parameters": {"action.$": "$.input.action", "topic.$": "$.input.topic", "ts.$": "$.input.ts", "filters.$": "$.input.filters"}, "ResultPath": "$.ConsumeMessages", "End": true}}}, "input_schema": {}, "globus_auth_scope": "https://auth.globus.org/scopes/0b23af30-5d20-4187-858d-aa921135475f/flow_0b23af30_5d20_4187_858d_aa921135475f_user", "synchronous": false, "log_supported": true, "types": ["Action"], "api_version": "1.0", "title": "Diapora-AP-Flow-a6c9", "subtitle": "", "description": "", "keywords": [], "principal_urn": "urn:globus:auth:identity:0b23af30-5d20-4187-858d-aa921135475f", "globus_auth_username": "0b23

In [40]:
ts_now = int(time.time() * 1000)
group_id = 'group-abc'
print(ts_now)
# print(group_id)

flow_input_b = {
    'input': {
        'action': 'consume',
        'topic': topic1,
        'ts': ts_now,
        # 'group_id': group_id,
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'university'}]}}},
            {'Pattern': {'value': {'content': [{'suffix': 'university'}]}}},
        ],
    },
}

1724815260290


In [41]:
# Get a client for the flow
specific_flow_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens[flow_id]['access_token'],
)
print(tokens[flow_id]['access_token'])
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    authorizer=specific_flow_authorizer,
)

# Run the flow
# Set a descriptive label for this flow run
run_label = f"Diaspora AP Flow by {primary_identity['preferred_username']}"
run = specific_flow_client.run_flow(
    body=flow_input_b,
    label=run_label,
    tags=['tutorial', 'diaspora'],
)

# Get run details
run_id = run['run_id']
run_status = run['status']
print('This flow can be monitored in the Web App:')
print(f'https://app.globus.org/runs/{run_id}')
print(f'Flow run started with ID: {run_id} - Status: {run_status}')

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')

# Run completed
print(json.dumps(run.data, indent=2))

AgW3XqdVa9kqX9JP7wQV7k6jV7XxrPnDBnwoPODG74Bx022z18igCko6j75VEOxx1xYy0nXpV3jV5xCl1lw4miBPMxO
This flow can be monitored in the Web App:
https://app.globus.org/runs/69c3c0bd-b58a-4de6-98b3-a429c6795918
Flow run started with ID: 69c3c0bd-b58a-4de6-98b3-a429c6795918 - Status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: SUCCEEDED
{
  "run_id": "69c3c0bd-b58a-4de6-98b3-a429c6795918",
  "flow_id": "0b23af30-5d20-4187-858d-aa921135475f",
  "flow_title": "Diapora-AP-Flow-a6c9",
  "flow_last_updated": "2024-08-28T03:20:53.732430+00:00",
  "start_time": "2024-08-28T03:21:15.683915+00:00",
  "completion_time": "2024-08-28T03:22:34.536000+00:00",
  "status": "SUCCEEDED",
  "display_status": "SUCCEEDED",
  "details": {
    "code": "FlowSucceede